In [1]:
                %%capture
pip install pandas 
pip install geopandas 
pip install requests 

SyntaxError: invalid syntax (<ipython-input-1-c75d43f3af4c>, line 1)

In [1]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import folium

/opt/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
centr_100 = gpd.read_file("/Users/panssies/Desktop/diplom/practice/sochi/centroids_100.gpkg", encoding='latin1')
centr_250 = gpd.read_file("/Users/panssies/Desktop/diplom/practice/sochi/centroids_250.gpkg", encoding='latin1')

display(centr_100.head(2))
display(centr_250.head(2))

,DN,id,heights_di,dispersion,median,qntl0,qntl25,qntl75,qntl1,geometry
0,131,1,52.0,194.609375,121.5,111.0,112.75,133.25,163.0,POINT (39.59750 43.81861)
1,121,2,36.0,105.225600,116.0,111.0,112.00,127.00,147.0,POINT (39.59778 43.81861)


,DN,id,heights_di,dispersion,median,geometry
0,131,1,150.0,1451.813414,133.0,POINT (39.59750 43.81861)
1,121,2,142.0,1188.038817,133.0,POINT (39.59778 43.81861)


In [3]:
centr_250.rename(columns={'median':'median250'}, inplace=True)
result = centr_100.merge(centr_250, on='id', how='right')

In [4]:
result.head(2)

,DN_x,id,heights_di_x,dispersion_x,median,qntl0,qntl25,qntl75,qntl1,geometry_x,DN_y,heights_di_y,dispersion_y,median250,geometry_y
0,131,1,52.0,194.609375,121.5,111.0,112.75,133.25,163.0,POINT (39.59750 43.81861),131,150.0,1451.813414,133.0,POINT (39.59750 43.81861)
1,121,2,36.0,105.225600,116.0,111.0,112.00,127.00,147.0,POINT (39.59778 43.81861),121,142.0,1188.038817,133.0,POINT (39.59778 43.81861)


In [5]:
result["complex_terrain"] = np.nan
for i in range(len(result)):
    if result['median250'][i] >= result['median'][i] and result['median250'][i] - result['median'][i]<=4:
        result.at[i,'complex_terrain'] = 1
    elif result['median250'][i] >= result['median'][i] and result['median250'][i] - result['median'][i]>4 and result['median'][i] - result['median250'][i]<=15:
        result.at[i,'complex_terrain'] = 2
    elif result['median250'][i] >= result['median'][i] and result['median250'][i] - result['median'][i]>15 and result['median'][i] - result['median250'][i]<=25:
        result.at[i,'complex_terrain'] = 3
    elif result['median250'][i] >= result['median'][i] and result['median250'][i] - result['median'][i]>25:
        result.at[i,'complex_terrain'] = 4
    elif result['median250'][i] <= result['median'][i] and result['median'][i] - result['median250'][i]<=4:
        result.at[i,'complex_terrain'] = 1
    elif result['median250'][i] <= result['median'][i] and result['median'][i] - result['median250'][i]>4 and result['median'][i] - result['median250'][i]<=15:
        result.at[i,'complex_terrain'] = 2
    elif result['median250'][i] <= result['median'][i] and result['median'][i] - result['median250'][i]>15 and result['median'][i] - result['median250'][i]<=25:
        result.at[i,'complex_terrain'] = 3
    elif result['median250'][i] <= result['median'][i] and result['median'][i] - result['median250'][i]>25:
        result.at[i,'complex_terrain'] = 4
    else:
        result.at[i,'complex_terrain'] = 5

print(result)


        DN_x      id  heights_di_x  dispersion_x  median  qntl0  qntl25  \
0        131       1          52.0    194.609375   121.5  111.0  112.75   
1        121       2          36.0    105.225600   116.0  111.0  112.00   
2        116       3          29.0     63.463018   113.5  109.0  112.00   
3        112       4          25.0     38.688866   113.0  108.0  111.00   
4        113       5          25.0     47.107143   112.5  108.0  111.00   
...      ...     ...           ...           ...     ...    ...     ...   
623945     7  623946           5.0      1.087370     6.0    NaN     NaN   
623946     5  623947           4.0      1.008310     6.0    NaN     NaN   
623947     8  623948           4.0      1.020833     6.0    NaN     NaN   
623948     9  623949           4.0      0.922495     6.0    NaN     NaN   
623949     7  623950           4.0      0.958678     6.0    NaN     NaN   

        qntl75  qntl1                 geometry_x  DN_y  heights_di_y  \
0       133.25  163.0  POIN

In [6]:
result.head(2)

,DN_x,id,heights_di_x,dispersion_x,median,qntl0,qntl25,qntl75,qntl1,geometry_x,DN_y,heights_di_y,dispersion_y,median250,geometry_y,complex_terrain
0,131,1,52.0,194.609375,121.5,111.0,112.75,133.25,163.0,POINT (39.59750 43.81861),131,150.0,1451.813414,133.0,POINT (39.59750 43.81861),2.0
1,121,2,36.0,105.225600,116.0,111.0,112.00,127.00,147.0,POINT (39.59778 43.81861),121,142.0,1188.038817,133.0,POINT (39.59778 43.81861),2.0


In [7]:
result_gdf = gpd.GeoDataFrame(
    result, geometry=result.geometry_x, crs="EPSG:4326"
)
del result_gdf['geometry_x']
del result_gdf['geometry_y']

In [8]:
result_gdf.head(2)

,DN_x,id,heights_di_x,dispersion_x,median,qntl0,qntl25,qntl75,qntl1,DN_y,heights_di_y,dispersion_y,median250,complex_terrain,geometry
0,131,1,52.0,194.609375,121.5,111.0,112.75,133.25,163.0,131,150.0,1451.813414,133.0,2.0,POINT (39.59750 43.81861)
1,121,2,36.0,105.225600,116.0,111.0,112.00,127.00,147.0,121,142.0,1188.038817,133.0,2.0,POINT (39.59778 43.81861)


In [11]:
print(type(result_gdf))
result_gdf.to_file("/Users/panssies/Desktop/diplom/practice/sochi/try2.shp")

<class 'geopandas.geodataframe.GeoDataFrame'>


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
